# Feature Engineering → Define enrichment parameters (team size, leadership, budget), test their extraction
Define - which parameters to extracts Sales and Sven (discuss feedback, send a another survey)
Test - with (our and external) NLPs and LLMs
For some params - agent should also have a search capacity
(external tools) OpenAI (DeepSeek) - ?/10 quality
(our own model) BERT
BERT - ?/10 quality
other...
Iterate (check if it works, get approved by Sven and scales)
Check which params work and do not work

## Importing Libraries

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
df_cv_sample = pd.read_excel('30_sample_users_cleaned.xlsx')
enrichment_params = pd.read_excel('ACP - Enrichment Parameters.xlsx')

/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/openpyxl/worksheet/_reader.py:329: UserWarning: Data Validation extension is not supported and will be removed
  warn(msg)


In [ ]:
df_cv_sample.head()

In [ ]:
# Check type of data in each column
df_cv_sample.dtypes

In [5]:
# Check for missing values
df_cv_sample.isnull().sum()

# Fill missing values with 0
df_cv_sample.fillna(0, inplace=True)

In [ ]:
# Print column names
df_cv_sample.columns

In [ ]:
df_cv_sample.experience[0]

In [ ]:
enrichment_params.head()

In [ ]:
import openai
import pandas as pd
import json
from tqdm import tqdm
import os

# New style client for OpenAI v1.x
client = openai.OpenAI(api_key=os.getenv("OPENAI_API_KEY"))

def enrich_experience(entry): #TODO: Implement websearch to get company type, industry, etc.
    # Skip non-dict entries
    if not isinstance(entry, dict):
        return {
            "company_type": "Unknown",
            "industry": "Unknown",
            "leadership_scope": "Unknown",
            "team_size_managed": "Unknown",
            "budget_responsibility": "Unknown"
        }

    company = entry.get('organization_name', 'Unknown')
    role = entry.get('title', 'Unknown')
    city = entry.get('city', 'Unknown')
    start = entry.get('start_date', 'Unknown')
    end = entry.get('end_date', 'Unknown')

    prompt = f"""
    You are an expert recruiter. Based on the following job experience details, classify:
    1. Company Type (Startup / PE-backed / Corporate / Unknown)
    2. Industry (e.g., Retail, Automotive, Consulting, etc.)
    3. Leadership Scope (Operational / Strategic / Both / Unknown)
    4. Team Size Managed (number or Unknown)
    5. Budget Responsibility (amount or Unknown)

    Company Name: {company}
    Job Title: {role}
    City: {city}
    Dates: {start} to {end}

    Return the answers in JSON format like this:
    {{
      "company_type": "PE-backed",
      "industry": "Consumer Goods",
      "leadership_scope": "Strategic",
      "team_size_managed": "10-20",
      "budget_responsibility": "€1M-€5M"
    }}
    """

    try:
        response = client.chat.completions.create(
            model='gpt-3.5-turbo',
            # model = 'gpt-4o-mini-search-preview',
            # model='gpt-4o',
            messages=[{"role": "user", "content": prompt}],
            temperature=0
        )

        content = response.choices[0].message.content.strip()

        # Convert JSON from model
        json_result = json.loads(content)
    except Exception as e:
        print(f"Error: {e}")
        json_result = {
            "company_type": "Unknown",
            "industry": "Unknown",
            "leadership_scope": "Unknown",
            "team_size_managed": "Unknown",
            "budget_responsibility": "Unknown"
        }

    return json_result

def process_experience_column(experience_column):
    enriched_profiles = []
    for exp_json in tqdm(experience_column):
        profile_results = []
        try:
            if pd.isna(exp_json) or not isinstance(exp_json, str):
                enriched_profiles.append([])
                continue

            # Fix common JSON issues
            exp_json_cleaned = (
                exp_json.replace("√§", "ä")
                        .replace("√º", "ü")
                        .replace("√ñ", "Ö")
                        .replace("'", '"')
            )
            experiences = json.loads(exp_json_cleaned)

            # Ensure it's a list of dictionaries
            if isinstance(experiences, dict):
                experiences = [experiences]
            elif not isinstance(experiences, list):
                enriched_profiles.append([])
                continue

            for entry in experiences:
                enriched = enrich_experience(entry)
                enriched_entry = {
                    "organization_name": entry.get('organization_name', ''),
                    "role": entry.get('title', ''),
                    **enriched
                }
                profile_results.append(enriched_entry)
                print(enriched_entry)

        except Exception as e:
            print(f"Error processing profile: {e}")
            profile_results.append({"error": str(e)})

        enriched_profiles.append(profile_results)

    return enriched_profiles

# Example DataFrame
df = df_cv_sample.copy().head(2)

# Process experiences
df['enriched_experience'] = process_experience_column(df['experience'])

# Save output
df.to_json('enriched_experience_output.json', orient='records')